In [2]:
#여기 코드가 가장 중요하다. 위의 코드를 사용하면 matplot창을 띄워주고
# 아래의 코드를 사용하면 파이썬에서 첫번째 이미지만 보여준다.


%matplotlib qt

#%matplotlib inline

In [3]:
import tensorflow as tf
import os

def download_model(model_type):
    server_prefix = 'https://omnomnom.vision.rwth-aachen.de/data/metrabs'
    model_zippath = tf.keras.utils.get_file(
        origin=f'{server_prefix}/{model_type}_20211019.zip',
        extract=True, cache_subdir='models')
    model_path = os.path.join(os.path.dirname(model_zippath), model_type)
    return model_path

model = tf.saved_model.load(download_model('metrabs_mob3l_y4t')) # or metrabs_eff2l_y4 for the big model
! wget -q https://raw.githubusercontent.com/isarandi/metrabs/master/img/test_image_3dpw.jpg


## area3 이미지 3장 분석


area2_145.jpg,230,135

In [ ]:
import cv2
image1 = cv2.imread('C:/Users/Leeyourack/jookyoung/image/area3/area3_233.jpg')
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
pred1 = model.detect_poses(image1, skeleton='smpl_24')

In [6]:
import cv2
image2 = cv2.imread('C:/Users/Leeyourack/jookyoung/image/area3/area3_203.jpg')
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
pred2 = model.detect_poses(image2, skeleton='smpl_24')

In [8]:
import cv2
image3 = cv2.imread('C:/Users/Leeyourack/jookyoung/image/area3/area3_245.jpg')
image3 = cv2.cvtColor(image3, cv2.COLOR_BGR2RGB)
pred3 = model.detect_poses(image3, skeleton='smpl_24')

### area3_normal vector

In [10]:
poses3d1=pred1['poses3d'].numpy()[0]
poses3d1[..., 1], poses3d1[..., 2] = poses3d1[..., 2], -poses3d1[..., 1]

In [11]:
a=poses3d1[10]
a

array([ 166.1, 5129.6, -494.7], dtype=float32)

In [12]:
poses3d1.T[0]

array([-2.1825000e+02, -1.8430000e+02, -1.5675000e+02, -2.3872501e+02,
        2.2187500e-01, -6.1328125e+00, -3.3870001e+02,  2.1010001e+02,
        2.2589999e+02, -3.8510001e+02,  1.6610001e+02,  2.1167500e+02,
       -5.0625000e+02, -4.6420001e+02, -4.2385001e+02, -5.8200000e+02,
       -4.9025000e+02, -4.0970001e+02, -3.9435001e+02, -2.7995001e+02,
       -3.8560001e+02, -2.9675000e+02, -3.7914999e+02, -2.8462500e+02],
      dtype=float32)

In [13]:
poses3d2=pred2['poses3d'].numpy()[0]
poses3d2[..., 1], poses3d2[..., 2] = poses3d2[..., 2], -poses3d2[..., 1]

In [14]:
b=poses3d2[10]
b

array([ 188.425, 7096.8  ,  685.   ], dtype=float32)

In [15]:
poses3d3=pred3['poses3d'].numpy()[0]
poses3d3[..., 1], poses3d3[..., 2] = poses3d3[..., 2], -poses3d3[..., 1]
 

In [16]:
c=poses3d3[10]
c

array([ -67.47813, 5987.2    , -177.4625 ], dtype=float32)

In [17]:
ab = b-a
print(ab)
ac = c-a
print(ac)

[  22.324997 1967.1997   1179.7     ]
[-233.57812  857.6001   317.23752]


In [18]:
list = [ab[1]*ac[2]-ab[2]*ac[1],-(ab[0]*ac[2]-ab[2]*ac[0]),ab[0]*ac[1]-ab[1]*ac[0]]
list

[-387641.25, -282634.4, 478640.72]

In [19]:
list = [(ab[1]*ac[2]-ab[2]*ac[1]),
        -(ab[0]*ac[2]-ab[2]*ac[0]),
            (ab[0]*ac[1]-ab[1]*ac[0])]
list

[-387641.25, -282634.4, 478640.72]

In [20]:
list[0]

-387641.25

### rotation matrix

In [21]:
x=list[0]
y=list[1]
z=list[2]
print('x:',x, 'y:',y, 'z:',z )
xy = (x**2+y**2)**0.5
yz = (y**2+z**2)**0.5
xz = (x**2+z**2)**0.5
print('xy:',xy, 'yz:',yz, 'xz:',xz )

xyz = (x**2+y**2+z**2)**0.5
print('xyz:',xyz)


x: -387641.25 y: -282634.4 z: 478640.72
xy: 479737.3722130188 yz: 555858.9256653226 xz: 615924.2456236636
xyz: 677676.0907272509


In [22]:
#z축 회전

import math
cos1 = math.cos(math.pi/2+abs(y/xy))
sin1 = math.sin(math.pi/2+abs(y/xy))


print(cos1,sin1)

-0.5556495544561977 0.8314166059399036


In [23]:
math.cos(math.pi/2)

6.123233995736766e-17

In [24]:
#y축 회전
import math
cos2 = math.cos(math.pi/2-abs(xy/xyz))
sin2 = math.sin(math.pi/2-abs(xy/xyz))
sin2 = -sin2

print(cos2,sin2)

0.6502515098837396 -0.7597190098279212


In [25]:
import math
math.cos(math.pi*1/2)

6.123233995736766e-17

In [26]:
import numpy as np

r1 = np.array([[cos1,-sin1,0], #z축 회전
                [sin1,cos1,0],
                [0,0,1]])
r2 = np.array([[cos2,0,sin2],
                [0,1,0],
                [-sin2,0,cos2]])        #y축 회전
r3 = np.array([[0,1,0], #z축 회전
                [-1,0,0],
                [0,0,1]])
                

In [27]:
import numpy as np

r1 = np.array([[cos1,-sin1,0],          #z축 회전
                [sin1,cos1,0],
                [0,0,1]])
r2 = np.array([[cos2,0,sin2],           #y축 회전
                [0,1,0],
                [-sin2,0,cos2]])        
r3 = np.array([[math.cos(math.pi*1/1.6),math.sin(math.pi*1/1.6),0],                 #z축 회전
                [-math.sin(math.pi*1/1.6),math.cos(math.pi*1/1.6),0],
                [0,0,1]])

In [28]:
r3

array([[-0.38268343,  0.92387953,  0.        ],
       [-0.92387953, -0.38268343,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [29]:
rotation1 = r3@r2@r1
rotation1

array([[ 0.90639689, -0.30646314,  0.29073188],
       [ 0.01563937,  0.71211478,  0.70188884],
       [-0.42213753, -0.631643  ,  0.65025151]])

## 변환행렬 적용 전

In [30]:
# 변환적용 전

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image1)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    '''a=np.ones(24)*(-6000)
    b=np.ones(24)
    c=np.ones(24)*(-2700)
    shift=np.row_stack((a,b,c))'''
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        #p1=pose3d.T+shift
        #pose3d=rotation1@pose3d.T
        #pose3d=pose3d+shift
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d[i_start], pose3d[i_end]), marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d.T, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image1, 
    pred1['boxes'].numpy(),
    pred1['poses3d'].numpy(),
    pred1['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

## 변환행렬 적용 후

In [31]:
##1
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image1)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    a=np.ones(24)*(1000)
    b=np.ones(24)*(+0)
    c=np.ones(24)*(+3500)
    shift=np.row_stack((a,b,c))
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        
        pose3d=rotation1@pose3d.T
        pose3d=pose3d+shift

        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d.T[i_start], pose3d.T[i_end]), marker='o', markersize=2)
        
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image1, 
    pred1['boxes'].numpy(),
    pred1['poses3d'].numpy(),
    pred1['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

In [32]:
##2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image2)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    a=np.ones(24)*(0)
    b=np.ones(24)*(+19000)
    c=np.ones(24)*(+2300)
    shift=np.row_stack((a,b,c))
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        
        pose3d=rotation1@pose3d.T
        pose3d=pose3d+shift

        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d.T[i_start], pose3d.T[i_end]), marker='o', markersize=2)
        
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image2, 
    pred2['boxes'].numpy(),
    pred2['poses3d'].numpy(),
    pred2['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

In [51]:
##3
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image3)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    a=np.ones(24)*(0)
    b=np.ones(24)*(+19000)
    c=np.ones(24)*(+2300)
    shift=np.row_stack((a,b,c))
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        
        pose3d=rotation1@pose3d.T
        pose3d=pose3d+shift
        print(pose3d)
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d.T[i_start], pose3d.T[i_end]), marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d, s=2)
        pose_ax.text(pose3d[0][14],pose3d[1][14],pose3d[2][14],"camera",color = 'blue')

    fig.tight_layout()
    plt.show()

visualize(
    image3, 
    pred3['boxes'].numpy(),
    pred3['poses3d'].numpy(),
    pred3['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

[[-1814.55286056 -1875.14391706 -1750.00278447 -1794.58464213
  -1942.36906373 -1690.35034878 -1812.03765136 -1879.13116624
  -1628.6159837  -1843.56264158 -1947.61216095 -1637.71918878
  -1888.95458988 -1928.99913414 -1796.19232859 -1934.00415971
  -2016.26092    -1728.40383898 -2022.75262541 -1627.16864819
  -2100.16265553 -1613.82932331 -2135.21386827 -1610.28781597]
 [23233.12556898 23187.20276948 23248.50267898 23225.45937036
  23199.36428377 23169.65689938 23276.91552206 23108.703112
  22938.92809845 23316.46713128 23137.95949654 22996.55467447
  23359.58735157 23281.38846472 23385.18260227 23426.87944415
  23214.14231741 23444.84963238 23105.60784848 23437.77067048
  23106.28523474 23493.15012285 23108.21469077 23528.92269011]
 [ -725.49895148  -812.46767163  -834.47182358  -607.04411873
  -1116.97143976 -1157.48437259  -455.70627784 -1487.97396406
  -1432.362111    -430.01702699 -1568.68330166 -1545.3581761
   -203.59498814  -292.59256934  -302.07064256  -169.19940941
   -291.8

In [49]:
pred3['poses3d'].numpy().T

array([[[-301.325    ],
        [-320.25     ],
        [-196.575    ],
        [-333.35     ],
        [-252.45     ],
        [  -7.3835936],
        [-412.25     ],
        [ -39.935158 ],
        [ 161.       ],
        [-451.05     ],
        [ -67.47813  ],
        [ 201.35     ],
        [-587.1      ],
        [-587.05     ],
        [-461.05     ],
        [-641.4      ],
        [-667.5      ],
        [-391.8      ],
        [-574.5      ],
        [-195.85     ],
        [-553.9      ],
        [ -89.9625   ],
        [-551.5      ],
        [ -44.75     ]],

       [[-476.3      ],
        [-369.9      ],
        [-435.       ],
        [-553.75     ],
        [-160.8875   ],
        [-186.9625   ],
        [-683.2      ],
        [ 125.60625  ],
        [ 135.775    ],
        [-718.5      ],
        [ 177.4625   ],
        [ 171.45     ],
        [-882.8      ],
        [-758.4      ],
        [-863.7      ],
        [-939.3      ],
        [-686.3      ],
        [-914.